In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
from requests.exceptions import HTTPError
import yaml

## Read CDSE credentials

In [ ]:
cdse_credentials_file = "credentials/credentials_cdse.yaml"

with open(cdse_credentials_file, "r") as file:
    cdse_credentials = yaml.safe_load(file)

## Set up access and refresh token

In [ ]:
cdse_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

data = {
    "grant_type": "password",
    "username": f"{cdse_credentials["login"]}",
    "password": f"{cdse_credentials["password"]}",
    "client_id": "cdse-public",
}

response = requests.post(
    url=cdse_token_url,
    data=data,
)

access_token = response.json()["access_token"]
refresh_token = response.json()["refresh_token"]

### Define function to refresh access token
The refresh token can be used to re-generate the access token without supplying usernames and passwords

In [ ]:
def refresh_cdse_access_token_header(refresh_token):

    cdse_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

    data = {
        "grant_type": "refresh_token",
        "refresh_token": f"{refresh_token}",
        "client_id": "cdse-public",
    }

    response = requests.post(
        url=cdse_token_url,
        data=data,
    )

    access_token = response.json()["access_token"]
    headers = {"Authorization": f"Bearer {access_token}"}

    return headers

## PULL subscription

### Create the subscription

It appears a user may only have one subscription active. 
The code will attempt to create a subscription, and return a message if it fails

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

# Create pull subscription
pull_subscription_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions"

pull_subscription_data = {
    "StageOrder": True,
    "FilterParam": "Collection/Name eq 'SENTINEL-1' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'IW_SLC__1S')",
    "Priority": 1,
    "Status": "running",
    "SubscriptionEvent": "modified"
}

# Try to create the subscription, raise an error if limit of subscriptions is reached
try:
    subscription_response = requests.post(
        url=pull_subscription_url, 
        json=pull_subscription_data, 
        headers=headers
    )
    subscription_response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    message = subscription_response.json()["detail"]["message"]
    print(f"HTTP error message: {message}")

### View the subscription

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

cdse_subscription_info_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions/Info"

subscription_info_response = requests.get(url=cdse_subscription_info_url, headers=headers).json()
subscription_info_response


In [ ]:
subscription_id = subscription_info_response[0]["Id"]
subscription_id

### Read subscription

In [ ]:
headers = refresh_cdse_access_token_header(refresh_token)

read_subscription_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Subscriptions({subscription_id})/Read?$top=2"
read_subscription_response = requests.get(url=read_subscription_url, headers=headers)

In [ ]:
read_subscription_response.json()


### Acknowledge subscription

### Delete subscription